In [8]:
# Kütüphane Yükleme
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from imblearn.over_sampling import SMOTE # Dengesizlik çözümü

# --- Veri Ön İşleme Kodları ---
print("1. Veri yükleniyor ve temizleniyor...")
try:
    df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
except FileNotFoundError:
    print("HATA: Veri dosyası 'data/WA_Fn-UseC_-Telco-Customer-Churn.csv' bulunamadı.")
    raise

df.drop('customerID', axis=1, inplace=True)
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# 2. İkili Özellikleri Dönüştürme
print("2. İkili (Binary) özellikler dönüştürülüyor...")
binary_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].map({'Yes': 1, 'No': 0, 'Female': 0, 'Male': 1})
        elif col == 'SeniorCitizen':
            # SeniorCitizen zaten sayısal (0/1), dokunmuyoruz
            pass

# 3. Çoklu Kategorik Özellikler (One-Hot Encoding)
print("3. Kategorik özellikler One-Hot Encoding ile dönüştürülüyor...")
categorical_cols = [
    'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies', 'Contract',
    'PaymentMethod'
]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# 4. Veri Bölme ve Ölçekleme
print("4. Veri bölme ve Standard Ölçekleme uygulanıyor...")
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Scaler'ı kaydet
if not os.path.exists('models'):
    os.makedirs('models')
with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# 5. SMOTE Uygulama (Eğitim verisini dengeler)
print("5. SMOTE uygulanarak eğitim verisi dengeleniyor...")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Modelin, yeni dengelenmiş veriyi kullanması için değişkenleri güncelleyelim.
X_train = X_train_smote
y_train = y_train_smote

print("\nVeri ön işleme ve dengeleme tamamlandı.")
print(f"Eğitim Verisi Boyutu (SMOTE Sonrası): {X_train.shape}")

1. Veri yükleniyor ve temizleniyor...
2. İkili (Binary) özellikler dönüştürülüyor...
3. Kategorik özellikler One-Hot Encoding ile dönüştürülüyor...
4. Veri bölme ve Standard Ölçekleme uygulanıyor...
5. SMOTE uygulanarak eğitim verisi dengeleniyor...

Veri ön işleme ve dengeleme tamamlandı.
Eğitim Verisi Boyutu (SMOTE Sonrası): (8278, 30)


In [9]:
# 2. Hücre: MLP Mimarisi ve Eğitimi (Nihai Optimizasyon)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# MLP Mimarisi (Nöron sayısı artırıldı)
input_dim = X_train.shape[1]

model = Sequential([
    # GİZLİ KATMAN 1: Nöron 128 -> 256
    Dense(units=256, activation='relu', input_dim=input_dim), 
    Dropout(0.2), 
    
    # GİZLİ KATMAN 2: Nöron 64 -> 128
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    
    # ÇIKIŞ KATMANI
    Dense(units=1, activation='sigmoid')
])

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

# Modeli Eğitme (Epoch sayısı 150 -> 250)
print("\nMLP Modeli Daha Büyük Yapı ve Daha Uzun Epoch ile Eğitiliyor (Epochs=250)...")
history = model.fit(
    X_train, # Dengelenmiş veri
    y_train, 
    epochs=250, # Epoch sayısı artırıldı
    batch_size=32,          
    validation_split=0.1,   
    verbose=1
)
print("Model Yeniden Eğitimi Tamamlandı!")

C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 256)                 │           7,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,961 (160.00 KB)

 Trainable params: 40,961 (160.00 KB)

 Non-trainable params: 0 (0.00 B)

None

MLP Modeli Daha Büyük Yapı ve Daha Uzun Epoch ile Eğitiliyor (Epochs=250)...
Epoch 1/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.7599 - loss: 0.4969 - val_accuracy: 0.6486 - val_loss: 0.7140
Epoch 2/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7703 - loss: 0.4716 - val_accuracy: 0.7717 - val_loss: 0.5619
Epoch 3/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7821 - loss: 0.4632 - val_accuracy: 0.8068 - val_loss: 0.4784
Epoch 4/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7823 - loss: 0.4572 - val_accuracy: 0.8623 - val_loss: 0.4230
Epoch 5/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7915 - loss: 0.4476 - val_accuracy: 0.7862 - val_loss: 0.5043
Epoch 6/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7958 - loss: 0.4396 - val_accuracy: 0.8382 - val_loss: 0.4021
Epoch 7/250
233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7995 - loss: 0.4353 - val_accuracy: 0.8333 - val_loss: 0.4487
Epoch 8/25

In [11]:
from sklearn.metrics import classification_report

# Yeni Eşik Değeri
THRESHOLD = 0.60 # Precision'ı (hassasiyeti) artırmak için eşiği yükseltiyoruz

# Değerlendirme (Model değişmedi, sadece tahmin eşiği değişti)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nModel Doğruluğu (Test Seti): {accuracy*100:.2f}%")

# Tahminleri alma ve YENİ EŞİK ile sınıflandırma
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > THRESHOLD).astype(int)

print(f"\n--- Sınıflandırma Raporu (Nihai Eşik: {THRESHOLD}) ---")
print(classification_report(y_test, y_pred))

# Modeli Kaydetme
model.save('models/mlp_churn_model.h5')
print("\nModel 'models/mlp_churn_model.h5' olarak kaydedildi.")


Model Doğruluğu (Test Seti): 73.67%
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step



--- Sınıflandırma Raporu (Nihai Eşik: 0.6) ---
              precision    recall  f1-score   support

           0       0.83      0.85      0.84      1035
           1       0.54      0.51      0.52       374

    accuracy                           0.76      1409
   macro avg       0.68      0.68      0.68      1409
weighted avg       0.75      0.76      0.75      1409


Model 'models/mlp_churn_model.h5' olarak kaydedildi.
